[Reference](https://mskadu.medium.com/async-programming-in-python-part-2-advanced-patterns-and-techniques-7f6b65061b74)

In [2]:
import asyncio

async def limited_operation(semaphore, task_id):
    """An operation that should be limited in concurrency"""
    async with semaphore:
        print(f"Task {task_id} started (semaphore acquired)")
        await asyncio.sleep(2)  # Simulate work
        print(f"Task {task_id} completed (semaphore released)")
        return f"Result {task_id}"

async def demo_semaphore():
    # Only allow 2 operations to run concurrently
    semaphore = asyncio.Semaphore(2)

    # Start 5 tasks, but only 2 will run at once
    tasks = [
        limited_operation(semaphore, i)
        for i in range(5)
    ]

    results = await asyncio.gather(*tasks)
    return results

asyncio.run(demo_semaphore())

In [3]:
import asyncio
import random

async def producer(queue, producer_id):
    """Produces work items and puts them in the queue"""
    for i in range(3):
        item = f"Item-{producer_id}-{i}"
        await queue.put(item)
        print(f"Producer {producer_id} created {item}")
        await asyncio.sleep(random.uniform(0.1, 0.5))

async def consumer(queue, consumer_id):
    """Consumes work items from the queue"""
    while True:
        try:
            # Wait for work, but timeout after 2 seconds
            item = await asyncio.wait_for(queue.get(), timeout=2.0)
            print(f"Consumer {consumer_id} processing {item}")
            await asyncio.sleep(random.uniform(0.2, 0.8))  # Simulate processing
            queue.task_done()  # Mark task as completed
        except asyncio.TimeoutError:
            print(f"Consumer {consumer_id} timed out, shutting down")
            break

async def demo_queue():
    queue = asyncio.Queue(maxsize=5)  # Limit queue size

    # Start producers and consumers
    producers = [producer(queue, i) for i in range(2)]
    consumers = [consumer(queue, i) for i in range(3)]

    # Run everything concurrently
    await asyncio.gather(*producers, *consumers)

asyncio.run(demo_queue())

In [4]:
import asyncio

# Shared resource
counter = 0
counter_lock = asyncio.Lock()

async def increment_counter(worker_id):
    """Safely increment a shared counter"""
    global counter

    for i in range(3):
        async with counter_lock:
            # Critical section - only one coroutine can execute this
            current = counter
            await asyncio.sleep(0.1)  # Simulate some processing
            counter = current + 1
            print(f"Worker {worker_id}: counter = {counter}")

async def demo_lock():
    # Start multiple workers that all increment the same counter
    workers = [increment_counter(i) for i in range(3)]
    await asyncio.gather(*workers)
    print(f"Final counter value: {counter}")

asyncio.run(demo_lock())

In [5]:
import asyncio

async def unreliable_operation():
    """Simulate an operation that might hang"""
    await asyncio.sleep(5)  # Takes too long
    return "Success"

async def with_timeout():
    try:
        # Don't wait more than 2 seconds
        result = await asyncio.wait_for(unreliable_operation(), timeout=2.0)
        return result
    except asyncio.TimeoutError:
        print("Operation timed out")
        return "Timeout fallback"

async def multiple_timeouts():
    """Handle timeouts in concurrent operations"""
    async def safe_operation(op_id):
        try:
            await asyncio.sleep(op_id)  # Different durations
            return f"Operation {op_id} succeeded"
        except asyncio.CancelledError:
            return f"Operation {op_id} was cancelled"

    # Wrap coroutines in tasks so we can cancel/check them
    tasks = [asyncio.create_task(safe_operation(i)) for i in [0.5, 3, 1]]

    try:
        # All operations must complete within 2 seconds
        results = await asyncio.wait_for(
            asyncio.gather(*tasks),
            timeout=2.0
        )
        return results
    except asyncio.TimeoutError:
        print("Some operations timed out")
        # Cancel remaining tasks
        for task in tasks:
            if not task.done():
                task.cancel()
        # Optionally, gather results from completed/cancelled tasks
        results = []
        for task in tasks:
            try:
                results.append(await task)
            except asyncio.CancelledError:
                results.append("Cancelled")
        return results

print(asyncio.run(with_timeout()))
print(asyncio.run(multiple_timeouts()))

In [6]:
import asyncio

async def might_fail(task_id):
    await asyncio.sleep(0.5)
    if task_id == 2:
        raise ValueError(f"Task {task_id} failed")
    return f"Task {task_id} succeeded"

async def handle_partial_failures():
    """Continue processing even if some operations fail"""
    tasks = [might_fail(i) for i in range(5)]

    # gather with return_exceptions=True
    results = await asyncio.gather(*tasks, return_exceptions=True)

    successes = []
    failures = []

    for i, result in enumerate(results):
        if isinstance(result, Exception):
            failures.append(f"Task {i}: {result}")
        else:
            successes.append(result)

    print(f"Successes: {successes}")
    print(f"Failures: {failures}")

    return successes, failures

asyncio.run(handle_partial_failures())

In [7]:
import asyncio
import random

async def unreliable_api_call():
    """Simulate an API that fails randomly"""
    if random.random() < 0.7:  # 70% failure rate
        raise ConnectionError("API unavailable")
    return "API success"

async def retry_with_backoff(max_retries=3):
    """Retry with exponential backoff"""
    for attempt in range(max_retries):
        try:
            result = await unreliable_api_call()
            print(f"Success on attempt {attempt + 1}")
            return result
        except ConnectionError as e:
            if attempt == max_retries - 1:
                print(f"Failed after {max_retries} attempts")
                raise

            wait_time = 2 ** attempt  # Exponential backoff
            print(f"Attempt {attempt + 1} failed, retrying in {wait_time}s...")
            await asyncio.sleep(wait_time)

# This might take several attempts
asyncio.run(retry_with_backoff())

In [8]:
import asyncio
import concurrent.futures
import requests  # Synchronous HTTP library
import time

def sync_http_call(url):
    """A blocking HTTP call using requests library"""
    response = requests.get(url)
    return {
        'url': url,
        'status': response.status_code,
        'length': len(response.content)
    }

async def async_http_calls(urls):
    """Run synchronous HTTP calls in a thread pool"""
    loop = asyncio.get_event_loop()

    with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
        # Submit all sync operations to thread pool
        futures = [
            loop.run_in_executor(executor, sync_http_call, url)
            for url in urls
        ]

        # Wait for all to complete
        results = await asyncio.gather(*futures)
        return results

async def mixed_sync_async():
    """Demonstrate mixing sync and async operations"""
    urls = [
        'https://httpbin.org/delay/1',
        'https://httpbin.org/delay/2',
        'https://httpbin.org/status/200'
    ]

    start_time = time.time()

    # These sync operations run concurrently in threads
    http_results = await async_http_calls(urls)

    # This is native async
    await asyncio.sleep(0.5)

    print(f"Completed in {time.time() - start_time:.2f}s")
    return http_results

# Run the mixed example
results = asyncio.run(mixed_sync_async())
for result in results:
    print(f"URL: {result['url']}, Status: {result['status']}")

In [10]:
import asyncio
import logging
from dataclasses import dataclass
from typing import List, Optional
import aiohttp

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

@dataclass
class ProcessingResult:
    item_id: str
    success: bool
    data: Optional[dict] = None
    error: Optional[str] = None

class AsyncDataProcessor:
    """Main application class demonstrating async architecture"""

    def __init__(self, max_concurrent_requests: int = 5):
        self.semaphore = asyncio.Semaphore(max_concurrent_requests)
        self.session: Optional[aiohttp.ClientSession] = None
        self.results_queue = asyncio.Queue()

    async def __aenter__(self):
        """Async context manager entry"""
        self.session = aiohttp.ClientSession()
        return self

    async def __aexit__(self, exc_type, exc_val, exc_tb):
        """Cleanup resources"""
        if self.session:
            await self.session.close()

    async def fetch_data(self, item_id: str) -> ProcessingResult:
        """Fetch data for a single item with rate limiting"""
        async with self.semaphore:
            try:
                url = f"https://jsonplaceholder.typicode.com/posts/{item_id}"

                async with self.session.get(url) as response:
                    if response.status == 200:
                        data = await response.json()
                        logger.info(f"Successfully fetched item {item_id}")
                        return ProcessingResult(item_id, True, data)
                    else:
                        error_msg = f"HTTP {response.status}"
                        logger.error(f"Failed to fetch item {item_id}: {error_msg}")
                        return ProcessingResult(item_id, False, error=error_msg)

            except Exception as e:
                error_msg = str(e)
                logger.error(f"Exception fetching item {item_id}: {error_msg}")
                return ProcessingResult(item_id, False, error=error_msg)

    async def process_item(self, result: ProcessingResult) -> ProcessingResult:
        """Process fetched data"""
        if not result.success:
            return result

        try:
            # Simulate processing time
            await asyncio.sleep(0.1)

            # Example processing: extract title length
            processed_data = {
                'original_title': result.data.get('title', ''),
                'title_length': len(result.data.get('title', '')),
                'word_count': len(result.data.get('body', '').split()),
                'processed_at': asyncio.get_event_loop().time()
            }

            result.data = processed_data
            logger.info(f"Processed item {result.item_id}")
            return result

        except Exception as e:
            error_msg = f"Processing error: {str(e)}"
            logger.error(f"Failed to process item {result.item_id}: {error_msg}")
            return ProcessingResult(result.item_id, False, error=error_msg)

    async def worker(self, item_ids: List[str]):
        """Worker that fetches and processes items"""
        for item_id in item_ids:
            # Fetch data
            fetch_result = await self.fetch_data(item_id)

            # Process data
            processed_result = await self.process_item(fetch_result)

            # Store result
            await self.results_queue.put(processed_result)

    async def result_collector(self, expected_count: int) -> List[ProcessingResult]:
        """Collect results as they become available"""
        results = []

        while len(results) < expected_count:
            try:
                result = await asyncio.wait_for(
                    self.results_queue.get(),
                    timeout=30.0
                )
                results.append(result)
                logger.info(f"Collected result {len(results)}/{expected_count}")

            except asyncio.TimeoutError:
                logger.error("Timeout waiting for results")
                break

        return results

    async def process_batch(self, item_ids: List[str],
                          batch_size: int = 10) -> List[ProcessingResult]:
        """Process items in batches with concurrent workers and result collection"""

        # Split items into batches
        batches = [
            item_ids[i:i + batch_size]
            for i in range(0, len(item_ids), batch_size)
        ]

        # Start result collector
        collector_task = asyncio.create_task(
            self.result_collector(len(item_ids))
        )

        # Start worker tasks for each batch
        worker_tasks = [
            asyncio.create_task(self.worker(batch))
            for batch in batches
        ]

        # Wait for all workers to complete
        await asyncio.gather(*worker_tasks)

        # Get collected results
        results = await collector_task

        return results

async def main():
    """Main application entry point"""
    # Items to process
    item_ids = [str(i) for i in range(1, 21)]  # Process items 1-20

    async with AsyncDataProcessor(max_concurrent_requests=3) as processor:
        logger.info(f"Starting processing of {len(item_ids)} items")

        start_time = asyncio.get_event_loop().time()
        results = await processor.process_batch(item_ids, batch_size=5)
        end_time = asyncio.get_event_loop().time()

        # Analyse results
        successful = [r for r in results if r.success]
        failed = [r for r in results if not r.success]

        print("\n=== Processing Complete ===")
        print(f"Total time: {end_time - start_time:.2f}s")
        print(f"Successful: {len(successful)}/{len(results)}")
        print(f"Failed: {len(failed)}/{len(results)}")

        if failed:
            print("\nFailures:")
            for failure in failed:
                print(f"  {failure.item_id}: {failure.error}")

        if successful:
            print("\nSample successful result:")
            sample = successful[0]
            print(f"  Item {sample.item_id}: {sample.data}")

if __name__ == "__main__":
    asyncio.run(main())